In [1]:
import sail.core
import json
import getpass

# password = getpass.getpass(prompt="Password: ")
password = "SailPassword@123"

sail.core.login("lbart@igr.com", password, 6200, "192.168.0.193")

sail.core.load_safe_objects("./SafeObjects")
dg_test_object_guid = "479F9891BBDE41C2B8A06CF2E29F32AC"

job_id = sail.core.run_job(dg_test_object_guid)
print(job_id)

#second_job_id = sail.core.run_job(dg_test_object_guid)
#print(second_job_id)

datasets = sail.core.get_datasets()
provision_ds = ""
if ( datasets is not None ):
    json_datasets = json.loads(datasets)
    provision_ds = list(json_datasets.keys())[0]
    print(provision_ds)
else:
    print("No loaded datasets")
    
digital_contracts = sail.core.get_digital_contracts()
provision_dc = ""
if ( digital_contracts is not None):
    json_digital_contracts = json.loads(digital_contracts)
    provision_dc = list(json_digital_contracts.keys())[0]
    print(provision_dc)
else:
    print("No digital contracts")

{4BE43257-8D6F-4BB8-9FDE-3CEF2FF01319}
{2014A98D-99DB-714C-8EE9-31CA6C2B5781}
{162E94F6-0D15-41A3-ABF5-A811F0F29B95}


In [7]:
job_status = sail.core.get_job_status(job_id)
print("{}:{}".format(job_id,job_status))
#job_status = sail.core.get_job_status(second_job_id)
#print("{}:{}".format(second_job_id,job_status))

{4BE43257-8D6F-4BB8-9FDE-3CEF2FF01319}:Job Done


In [3]:
first_provision = sail.core.provision_secure_computational_node(provision_dc, provision_ds, "Standard_D8s_v4")
#second_provision = sail.core.provision_secure_computational_node(provision_dc, provision_ds, "Standard_D8s_v4")

print(first_provision)
#print(second_provision)

{"SCNGuid":"{42AAA140-018C-41A6-91CB-38482F70E4D0}","Status":true}


In [ ]:
print(sail.core.wait_for_all_secure_nodes_to_be_provisioned(30000))

In [4]:
my_table_guid = "2534C8FA-0E1A-D442-95A6-8ECB11A39746"
set_ds_response = sail.core.set_parameter(job_id, "4DC7BEC15B384FBD9BD721B7BF36A984", my_table_guid)
#set_ds_response = sail.core.set_parameter(second_job_id, "4C8041B4513A44FAA5409662E014519C", provision_ds)

#print("First job IP {}".format(sail.core.get_ip_for_job(job_id)))
#print("Second job IP {}".format(sail.core.get_ip_for_job(second_job_id)))

In [5]:
data_id = "{}.{}".format(job_id, "52A2698D65C44433BEDDFEE508C9065F")

pull_data_result = sail.core.pull_data(data_id)
print(pull_data_result)

Success


In [6]:
import base64
import pickle

message_response = sail.core.wait_for_data(10000)
while None != message_response:
    if ( message_response is not None):
        message_json = json.loads(message_response)
        if ( message_json["SignalType"] == 0):
            print("Job {} is running".format(message_json["JobUuid"]))

        if ( message_json["SignalType"] == 1):
            print("Job {} is done".format(message_json["JobUuid"]))

        if ( message_json["SignalType"] == 3):
            data_value = message_json["FileData"]
            decoded_value = base64.b64decode(data_value)
            un_serialized_value = pickle.loads(decoded_value)
            print("Data received from job {}: {}".format(message_json["JobUuid"], un_serialized_value))
            
    message_response = sail.core.wait_for_data(500)

Job 4BE432578D6F4BB89FDE3CEF2FF01319 is running
Job 4BE432578D6F4BB89FDE3CEF2FF01319 is done
Data received from job 4BE432578D6F4BB89FDE3CEF2FF01319:      Line Number  AGE   BMI  PD-L1 level before treatment  \
0              0   44  25.4                         86.89   
1              1   47  23.9                         80.59   
2              2   54  24.0                         86.68   
3              3   55  22.5                         89.55   
4              4   50  24.4                         89.81   
..           ...  ...   ...                           ...   
295          295   43  31.2                         83.27   
296          296   52  28.5                         81.01   
297          297   52  27.5                         82.89   
298          298   59  22.6                         86.65   
299          299   40  26.2                         87.71   

     PD-L1 level after treatment  PD-L2 level before treatment  \
0                          32.49                   

In [ ]:
safe_functions = sail.core.get_safe_functions()
if ( safe_functions is not None):
    json_safe_functions = json.loads(safe_functions)
    print(json.dumps(json_safe_functions,indent=4))
else:
    print("No safe functions")

In [ ]:
sail.core.exit_current_session()

In [ ]:
sail.core.deprovision_digital_contract(provision_dc)

In [ ]:
tables = sail.core.get_tables()
if ( tables is not None):
    json_tables = json.loads(tables)
    print(json.dumps(json_tables, indent=4))
else:
    print("No tables")


In [ ]:
job_id = sail.core.run_job(dg_test_object_guid)
print(job_id)

In [ ]:
digital_contracts = sail.core.get_digital_contracts()
provision_dc = ""
if ( digital_contracts is not None):
    json_digital_contracts = json.loads(digital_contracts)
    provision_dc = list(json_digital_contracts.keys())[0]
    print(provision_dc)
    print(json.dumps(json_digital_contracts, indent=4))
else:
    print("No digital contracts")

In [ ]:
datasets = sail.core.get_datasets()
provision_ds = ""
if ( datasets is not None ):
    json_datasets = json.loads(datasets)
    provision_ds = list(json_datasets.keys())[0]
    print(provision_ds)
    print(json.dumps(json_datasets, indent=4))
else:
    print("No loaded datasets")